In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import plotly.express as px
from plotly import graph_objects as go
from matplotlib.ticker import StrMethodFormatter
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler


sns.set_theme(style="dark")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [12]:
df = pd.read_csv('./train.csv')

In [13]:
df.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,players_in_match,kills_normalized,damage_dealt_normalized,heals_and_boosts,total_distance,total_items_acquired,team_size,max_team_size,match_size,team_indicator_1,team_indicator_2,team_indicator_3,team_indicator_4,team_indicator_5,game_mode
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.00,0,0.00,0,0,244.80,1,1466,0.44,96,0.00,0.00,0,244.80,1,4,8,96,0.02,0.06,0.19,0.42,0.31,custom
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.00,0,11.04,0,0,1434.00,5,0,0.64,91,0.00,99.70,0,1445.04,5,4,10,91,0.04,0.02,0.23,0.35,0.35,custom
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.00,0,0.00,0,0,161.80,2,0,0.78,98,0.00,69.36,0,161.80,2,2,6,98,0.06,0.69,0.18,0.00,0.06,duo
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.00,0,0.00,0,0,202.70,3,0,0.17,91,0.00,35.86,0,202.70,3,1,8,91,0.05,0.18,0.26,0.13,0.37,custom
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.00,0,0.00,0,0,49.75,2,0,0.19,97,1.03,103.00,0,49.75,2,1,2,97,0.96,0.04,0.00,0.00,0.00,solo


In [23]:
x = df[df.columns.difference(['Id', 'groupId', 'matchId','matchType', 'game_mode', 'winPlacePerc'])]

In [24]:
y = df.winPlacePerc

In [27]:
ss = StandardScaler()
xs = ss.fit_transform(x)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(xs, y, test_size = 0.3, random_state=20)

In [29]:
ln = LinearRegression()

ln_scores = cross_val_score(ln, x_train, y_train, cv=10)

print(ln_scores)
print(np.mean(ln_scores))

[0.84646196 0.84669356 0.84509849 0.84599192 0.84575455 0.84410344
 0.84566515 0.8450713  0.84652022 0.84527611]
0.8456636715294259


In [42]:
optimal_lasso = LassoCV(n_alphas=100, cv=10, verbose=1)
optimal_lasso.fit(x_train, y_train,)

print(optimal_lasso.alpha_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

0.0002492761995730044


In [41]:
lasso = LassoCV(n_alphas=1, cv=10)
lasso.fit(x_train, y_train)
lasso_scores = cross_val_score(lasso, x_train, y_train, cv=10)
print(lasso_scores)
print(np.mean(lasso_scores))

[0.84593211 0.84604877 0.84441876 0.84535457 0.84520817 0.84360373
 0.8449531  0.84443643 0.8459222  0.844663  ]
0.8450540842032002


In [44]:
ridge_alphas = np.logspace(0, 5, 20)
print(ridge_alphas)
ridge_alphas
optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=10)
optimal_ridge.fit(x_train, y_train)

print(optimal_ridge.alpha_)

[1.00000000e+00 1.83298071e+00 3.35981829e+00 6.15848211e+00
 1.12883789e+01 2.06913808e+01 3.79269019e+01 6.95192796e+01
 1.27427499e+02 2.33572147e+02 4.28133240e+02 7.84759970e+02
 1.43844989e+03 2.63665090e+03 4.83293024e+03 8.85866790e+03
 1.62377674e+04 2.97635144e+04 5.45559478e+04 1.00000000e+05]
6.158482110660264


In [48]:
ridge = Ridge(alpha=optimal_ridge.alpha_)

ridge_scores = cross_val_score(ridge, x_train, y_train, cv=10)

print(ridge_scores)
print(np.mean(ridge_scores))

[0.84646264 0.84669379 0.8450984  0.84599237 0.84577821 0.8441019
 0.84566682 0.84507275 0.84660195 0.84527592]
0.8456744748962384
